In [ ]:
import numpy as np
import pandas as pd 

In [ ]:
# Read data
df = pd.read_parquet('../../data/binance_data_BTCFDUSD_20240528183200.parquet',engine='pyarrow')

In [ ]:
# Check duplicate indexes
duplicate_indexes = df.index[df.index.duplicated()]
print(f"Duplicated indexes : {duplicate_indexes}")
if len(duplicate_indexes)>0:
    df = df[~df.index.duplicated(keep='last')]

In [ ]:
# Verify NaN in DataFrame
if df.isnull().values.any():
    print("NaN value in the DataFrame.")
    # Print lines with NaN
    nan_rows = df[df.isnull().any(axis=1)]
    #print(f"Lines with NaN : \n{nan_rows}")
    # Number of NaN by column
    nan_count = df.isnull().sum()
    print(f"Number of Nan by column : \n{nan_count}")
else:
    print("No NaN value in the DataFrame.")

In [ ]:
# Datetime index
df.index = pd.to_datetime(df.index)

In [ ]:
# Verify if each second if present
all_seconds = pd.date_range(start=df.index.min(),end=df.index.max(),freq='s')
missing_indexes = all_seconds.difference(df.index)

if len(missing_indexes) ==0:
    print("Every seconds are present")
else:
    print(f"It misses {len(missing_indexes)} second(s)")
    print(missing_indexes)

In [ ]:
# Include the missing seconds and copy the data from the previous seconds
df_full = df.reindex(all_seconds)
df_full.fillna(method='ffill',inplace=True)

#df_marked = df.copy()
#df_marked.fillna(-9999, inplace=True)
# Row with only NaN
#new_rows = df_full[df_full.isnull().all(axis=1)]
#print(new_rows)
# Copy data only for the row with only NaN
#df_full.loc[new_rows.index] = df_full.loc[new_rows.index].fillna(method='ffill')

In [ ]:
# Verify NaN in DataFrame
if df_full.isnull().values.any():
    print("NaN value in the DataFrame.")
    # Print lines with NaN
    nan_rows = df_full[df_full.isnull().any(axis=1)]
    print(f"Lines with NaN : \n{nan_rows}")
    # Number of NaN by column
    nan_count = df_full.isnull().sum()
    print(f"Number of Nan by colomn : \n{nan_count}")
else:
    print("No NaN value in the DataFrame.")

In [ ]:
df_full.to_parquet('../../data/data_cleaned.parquet', engine='pyarrow')